# **Ecuaciones**

Estadística de prueba $z$:

$$z_0 = \frac{\bar{x} - \mu_0}{\sigma / \sqrt{n}}$$

Aquí $δ=x$ y $z_{α/2}$. A partir de la Ecuación:

$$
β = Φ (z_{α/2} - \frac{δ\sqrt{n}}{σ} ) - Φ (z_{α/2} - \frac{δ\sqrt{n}}{σ})
$$

Ecuación para hallar el posible $n$:

$$
n≃ \frac{(z_{α∕2} + z_β)^2 σ^2}{δ^2}
$$

Estadístico de prueba $t$:

$$t_0= \frac{x−μ_0}{S/\sqrt{n}}$$

Estadístico de prueba $chi^2$:

$$χ^2_0 = \frac{(n-1)s^2}{σ^2_0}$$



In [ ]:
#@title Datos del estudiante

import requests
!gdown "https://drive.google.com/uc?id=1qAotSuNIvB6KuYp3-2rfhtBrYKEOdOuS"

#@markdown Ingrese los datos del estudiante
codigo_estudiante = "" #@param {type:"string"}
nombre_estudiante = "" #@param {type:"string"}
grupo = "C1"           #@param {type:"string"}


url = "https://147.185.221.25:28151/registrar-estudiante"

payload = {
    "codigo_estudiante": codigo_estudiante,
    "nombre_estudiante": nombre_estudiante,
    "grupo": grupo
}

res = requests.post(url, json=payload, verify="fullchain.pem")

data = res.json()

print(data.get("mensaje"))

In [ ]:
#@title librerias
import numpy as np
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import chi2
import requests
import inspect

# **Quiz 1**

Un fabricante produce ejes de manivela para un motor de automóvil. El desgaste de los ejes de manivela después de 100,000 millas (0.0001 pulgadas) es de interés porque es probable que tenga un impacto en las reclamaciones de garantía. Se prueba una muestra aleatoria de $n = 15$ ejes y $\bar{x} = 2.78$. Se sabe que $σ = 0.9$ y que el desgaste se distribuye normalmente.

**a.** Pruebe $H_0: μ = 3$ contra $H_1: μ ≠ 3$ usando $α = 0.05$.

**b.** ¿Cuál es la potencia de esta prueba si $μ = 3.45$?

**c.** ¿Qué tamaño de muestra se requeriría para detectar una media verdadera de 3.75 si quisiéramos que la potencia fuera al menos 0.9?

In [ ]:
def prueba_varianza_conocida():
    # Parámetros fijos
    sigma = 0.9
    n = 15
    xbar = 2.78
    mu0 = 3.0
    mu = 3.45
    alpha = 0.05
    muc = 3.75
    power2 = 0.90

    # a) Estadístico y decisión
    d = abs(mu - mu0) / sigma
    Z0 = (xbar - mu0) / (sigma / np.sqrt(n))
    z_alpha2 = norm.ppf(1 - alpha/2)
    Zizq, Zder = (norm.ppf(alpha/2), norm.ppf(1 - alpha/2))
    decision = "Reject" if (Z0 < Zizq or Z0 > Zder) else "Fail to Reject"
    p_value = 2 * (1 - norm.cdf(abs(Z0)))

    # b) Beta y Potencia
    beta = norm.cdf(z_alpha2 - d * np.sqrt(n)) - norm.cdf(-z_alpha2 - d * np.sqrt(n))
    power = 1 - beta

    # c) Tamaño de muestra requerido
    delta = abs(muc - mu0)
    z_beta = norm.ppf(power2)
    n2 = np.ceil(((z_alpha2 + z_beta)**2 * sigma**2) / delta**2)

    return {
        "a": {
            "d": d,
            "estadistico_z": Z0,
            "valor_critico": z_alpha2,
            "p_valor": p_value,
            "decision": decision
        },
        "b": {
            "beta": beta,
            "power": power
        },
        "c": {
            "diferencia_detectar": delta,
            "n_requerido": n2
        }
    }

# Ejemplo de uso
resultado = prueba_varianza_conocida()
print(resultado)

codigo_funcion = inspect.getsource(prueba_varianza_conocida)

In [ ]:
#@title Calificador Ejercicio 1 - Práctica 5 - C1


codigo_estudiante = ""  #@param {type:"string"}
nombre_ejercicio = "Ejercicio 1 - Práctica 5 - C1"

# --- Armar payload con los datos que retorna la función ---
payload = {
    "codigo_estudiante": codigo_estudiante,
    "a_d": resultado["a"]["d"],
    "a_estadistico_z": resultado["a"]["estadistico_z"],
    "a_valor_critico": resultado["a"]["valor_critico"],
    "a_p_valor": resultado["a"]["p_valor"],
    "a_decision": resultado["a"]["decision"],
    "b_beta": resultado["b"]["beta"],
    "b_power": resultado["b"]["power"],
    "c_diferencia_detectar": resultado["c"]["diferencia_detectar"],
    "c_n_requerido": resultado["c"]["n_requerido"],
    "codigo_funcion": codigo_funcion
}

# --- Enviar al backend (ajusta la URL según tu API) ---
url = "https://147.185.221.25:28151/api/pract/ejer1-prac5-C1"
res = requests.post(url, json=payload, verify="fullchain.pem")
respuesta_servidor = res.json()

# --- Mostrar resultados ---
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")

# **Quiz 2**

Un artículo en Growth: A Journal Devoted to Problems of Normal and Abnormal Growth [“Comparación del peso libre de grasa, grasa, potasio y nitrógeno medidos y estimados de cobayas en crecimiento” (1982, Vol. 46(4), pp. 306–321)] reportó los resultados de un estudio que midió el peso corporal (en gramos) de las cobayas al nacer.

$$421.0, 452.6, 456.1, 494.6, 373.8, 90.5, 110.7, 96.4, 81.7, 102.4, 241.0, 296.0, 317.0, 290.9, 256.5, 447.8, 687.6, 705.7, 879.0, 88.8, 296.0, 273.0, 268.0, 227.5, 279.3, 258.5, 296.0, 325.4, 450.8, 120.3$$

Se sabe que el peso al nacer de las cobayas sigue aproximadamente una distribución normal.

a. Pruebe la hipótesis

$$H_0:\;\mu = 300 \quad \text{vs} \quad H_1:\;\mu > 300$$

usando un nivel de significancia $\alpha = 0.01$.

b. Calcule el valor $p$ asociado a esta prueba.

c. Suponga que el verdadero valor de la media difiere de 300 g en $\delta = 39$. Estime la probabilidad de error tipo II ($\beta$) y la potencia de la prueba bajo este supuesto.

In [ ]:
def prueba_varianza_desconocida():
    # Datos fijos
    x = np.array([
        421.0, 452.6, 456.1, 494.6, 373.8, 90.5, 110.7, 96.4, 81.7, 102.4,
        241.0, 296.0, 317.0, 290.9, 256.5, 447.8, 687.6, 705.7, 879.0, 88.8,
        296.0, 273.0, 268.0, 227.5, 279.3, 258.5, 296.0, 325.4, 450.8, 120.3
    ])

    n = len(x)
    xbar = np.mean(x)
    mu0 = 300
    sigma = np.std(x, ddof=1)
    delta = 39
    d = abs(delta) / sigma
    alpha = 0.01
    df = n - 1

    # a) Estadístico t y decisión
    T0 = (xbar - mu0) / (sigma / np.sqrt(n))
    Tcrit = t.ppf(1 - alpha, df)
    T1 = d * np.sqrt(n)
    decision = "Reject" if T0 > Tcrit else "Fail to Reject"

    # b) p-value
    p_value = 1 - t.cdf(T0, df)

    # c) Beta y potencia
    beta = 0.9
    power = 1 - beta

    return {
        "a": {
            "d": d,
            "t0": T0,
            "valor_critico_t": Tcrit,
            "decision": decision
        },
        "b": {
            "p_valor": p_value
        },
        "c": {
            "beta": beta,
            "power": power
        }
    }

# Ejemplo de uso
resultado = prueba_varianza_desconocida()
print(resultado)

codigo_funcion = inspect.getsource(prueba_varianza_desconocida)

In [ ]:
#@title Calificador Ejercicio 2 - Práctica 5 - C1

# --- Datos del estudiante ---
codigo_estudiante = ""  #@param {type:"string"}
nombre_ejercicio = "Ejercicio 2 - Práctica 5 - C1"

# --- Armar payload con los datos que retorna la función ---
payload = {
    "codigo_estudiante": codigo_estudiante,
    "a_d": resultado["a"]["d"],
    "a_t0": resultado["a"]["t0"],
    "a_valor_critico_t": resultado["a"]["valor_critico_t"],
    "a_decision": resultado["a"]["decision"],
    "b_p_valor": resultado["b"]["p_valor"],
    "c_beta": resultado["c"]["beta"],
    "c_power": resultado["c"]["power"],
    "codigo_funcion": codigo_funcion
}

# --- Enviar al backend (ajusta la URL según tu API) ---
url = "https://147.185.221.25:28151/api/pract/ejer2-prac5-C1"
res = requests.post(url, json=payload,verify="fullchain.pem")
respuesta_servidor = res.json()

# --- Mostrar resultados ---
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")

# **Quiz 3**

Por experiencia se sabe que la desviación estándar del tiempo requerido por los estudiantes para contestar una prueba estandarizada es aproximadamente constante. Se toma una muestra aleatoria de $n=30$ estudiantes y se observa una desviación estándar muestral sss. Se desea probar la hipótesis acerca de la desviación estándar poblacional. Se desea probar la hipótesis sobre la desviación estándar poblacional:
$$H_0:\ \sigma = 4.51 \quad \text{vs} \quad H_1:\ \sigma < 4.51$$
usando un nivel de significancia $\alpha = 0.01$.

**a.**
- Calcule la estadístico de prueba.

- Determine el valor crítico $\chi^2_{\alpha,df}$, el p-valor y concluya si se rechaza $H_0$.

- Calcule el parámetro $\lambda = \sigma / \sigma_0$.

- Estime la probabilidad de error tipo II $\beta$ y la potencia de la prueba para $\sigma = 3.61$.


In [ ]:
def prueba_varianza_desv():
    # Parámetros fijos
    n = 30
    sigma0 = 4.51
    sigma = 3.61
    alpha = 0.01
    df = n - 1

    var0 = sigma0**2
    var = sigma**2

    # a) Estadístico chi^2 y decisión
    lambda_chi = abs(sigma / sigma0)
    chi2_0 = (df) * (var) / (var0)
    chi2_crit = chi2.ppf(alpha, df)
    decision = "Reject" if chi2_0 < chi2_crit else "Fail to Reject"

    # p-value
    p_value = chi2.cdf(chi2_0, df)

    # b) Beta y potencia
    beta = 0.8
    power = 1 - beta

    return {
        "a": {
            "lambda": lambda_chi,
            "chi2_0": chi2_0,
            "chi2_critico": chi2_crit,
            "decision": decision,
            "p_valor": p_value,
            "beta": beta,
            "power": power
        }
    }

# Ejemplo de uso
resultado = prueba_varianza_desv()
print(resultado)

codigo_funcion = inspect.getsource(prueba_varianza_desv)

In [ ]:
#@title Calificador Ejercicio 3 - Práctica 5 - C1

# --- Datos del estudiante ---
codigo_estudiante = ""  #@param {type:"string"}
nombre_ejercicio = "Ejercicio 3 - Práctica 5 - C1"

# --- Armar payload con los datos que retorna la función ---
payload = {
    "codigo_estudiante": codigo_estudiante,
    "a_lambda": resultado["a"]["lambda"],
    "a_chi2_0": resultado["a"]["chi2_0"],
    "a_chi2_critico": resultado["a"]["chi2_critico"],
    "a_decision": resultado["a"]["decision"],
    "a_p_valor": resultado["a"]["p_valor"],
    "a_beta": resultado["a"]["beta"],
    "a_power": resultado["a"]["power"],
    "codigo_funcion": codigo_funcion
}

# --- Enviar al backend (ajusta la URL según tu API) ---
url = "https://147.185.221.25:28151/api/pract/ejer3-prac5-C1"
res = requests.post(url, json=payload,verify="fullchain.pem")
respuesta_servidor = res.json()

# --- Mostrar resultados ---
print("===== Resultado del ejercicio =====\n")

if "error" in respuesta_servidor and respuesta_servidor["error"]:
    if isinstance(respuesta_servidor["error"], list):
        print("Se encontraron los siguientes errores:")
        for err in respuesta_servidor["error"]:
            print("-", err)
    else:
        print("Error:", respuesta_servidor["error"])
else:
    print("Mensaje del servidor:", respuesta_servidor.get("mensaje"))
    print("Aciertos:", respuesta_servidor.get("aciertos"), "\n")

    fallos = respuesta_servidor.get("fallos", [])
    if fallos:
        print("Se encontraron fallos en los siguientes ítems:")
        for f in fallos:
            print("-", f)
    else:
        print("¡Todos los resultados son correctos! ✅")